## **INTRODUCTION**

The purpose of this notebook is to use Python to build a data Pipeline. The data will be sourced from the Spotify API and assembled into a Pandas dataframe which will then be stored into a PostgreSQL database. The resultant of this exercise will be a series of highly informative Python visualizations that will be used to track the different KPI's of our data. 

Spotify is a music/podcast streaming platform containing numerous artists with their respective albums and tracks. Each artist and track has a popularity rating and this exercise is centered around analysing the factors affecting this measure.

## **PART 1: Make Spotify API Call**

**IMPORT LIBRARIES**

In [1]:
#import libraries
import requests
import numpy as np
import pandas as pd
import psycopg2 as ps
import datetime
import base64
import time

**AUTHENTICATION**

In [2]:
def authentication():
    
    ############ CLIENT ID AND CLIENT SECRET ################
    
    client_id = '65c95b1fcf354742978c546b96be30c3'
    client_secret = 'd83aca0f5c5542b1a2615b7c83fea2e7'
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())
    
    ############## API CALL PARAMETERS ######################
    
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_data = {
        'grant_type': 'client_credentials'
    }
    auth_headers = {
        'Authorization': f'Basic {client_creds_b64.decode()}'
    }
    
    ############ API CALL ###################################
    
    r = requests.post(auth_url, data=auth_data, headers=auth_headers)
    request_status = r.status_code in range(200,299)
    
    ########################################################
    
    if request_status == True:
        auth_response = r.json()
        now = datetime.datetime.now()
        access_token = auth_response['access_token']
        expires_in = auth_response['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
    return access_token, expires

In [3]:
token, expires = authentication()
print(token)

BQAmP8VmiZNsF3joPqcIyPOTY7c8mMCoWpTzA8osLOMQW-kqg0cb0o93Tn8LQALwTkMwy9pBgO-9bw0xfMM14zu9NpZDK9HOHeDuv8gAu6zsibYtjac


**ARTISTS**

In [4]:
artist_list = ['drake','young thug','adele','taylor swift','kanye west','mac miller','juice wrld','doja cat','lil baby','the weeknd','morgan wallen','j. cole','rod wave','billie eilish','dababy','pop smoke','future','eminem','bad bunny','rod wave','billie eilish','ariana grande','luke combs','roddy ricch','travis scott','Xxxtentacion','justin bieber','gunna','lil wayne','polo g','chris brown','morgan wallen','megan thee stallion','khalid','the beatles','moneybagg yo','harry styles','ed sheeran','kevin gates','bts','halsey','lil durk','kendrick lamar','jason aldean','beyonce','rihanna','ynw melly','kodak black','nf','tory lanez','maroon 5','lil peep','kane brown','dua lipa','luke bryan','imagine dragons','nle choppa','lady gaga','summer walker','21 savage','cardi b','queen','thomas rhett','florida georgia line','meek mill','lewis capaldi','frank ocean','fleetwood mac','nicki minaj','machine gun kelly','sam smith','one direction','blackbear','bruno mars','chris stapleton','blake shelton','lil tjay','migos','jack harlow',
'russ','j. balvin','shawn mendes','lil mosey','sam hunt','kehlani','tyler, the creator','twenty one pilots','kid cudi','logic','kenny chesney','don toliver','selena gomez','dj khaled','ozuna','marshmello','wiz khalifa','frank sinatra','nav','eagles','george strait']
len(artist_list)

100

In [5]:
artists = artist_list

**MAIN API CALL CLASS**

In [6]:
class Artist_breakdown():

  def __init__(self, artist_name, token):
    self.artist_name = artist_name
    self.token = token
    self.artist_json_file = self.artist_search()
    self.genre = self.artist_json_info()[0]
    self.artist_id = self.artist_json_info()[1]
    self.artist_name = self.artist_json_info()[2]
    self.artist_popularity = self.artist_json_info()[3]

    self.album_json_file = self.album_search()
    self.album_type = self.album_info()[0]
    self.total_tracks = self.album_info()[1]
    self.release_date = self.album_info()[2]
    self.album_id = self.album_info()[3]
    self.available_markets = self.album_info()[4]
    self.album_name = self.album_info()[5]

    #self.track_json_file = self.track_search() ####
    self.duration_ms = [] #self.track_info()[1]
    self.track_id = [] #self.track_info()[2]
    self.track_title = self.track_info()[0]

    #self.track_deep_json_file = self.track_deep_search() ####
    self.track_popularity = self.track_deep_info()

  def artist_search(self):  
    print(self.artist_name)
    token_url = 'https://api.spotify.com/v1/search'
    query = {'q': self.artist_name,'type':'artist'}
    authorization = {'Authorization': f'Bearer {self.token}'}
    time.sleep(0.3)
    artist_response = requests.get(token_url, params = query, headers = authorization).json()
    return artist_response

  def artist_json_info(self):
    artist_search_response = self.artist_json_file
    genre = artist_search_response['artists']['items'][0]['genres'][0]
    artist_id = artist_search_response['artists']['items'][0]['id']
    artist_name = artist_search_response['artists']['items'][0]['name']
    artist_popularity = artist_search_response['artists']['items'][0]['popularity']
    return genre,artist_id,artist_name,artist_popularity

  def album_search(self):
    token_url = f'https://api.spotify.com/v1/artists/{self.artist_id}/albums'
    query = {'id':self.artist_id}
    authorization = {'Authorization': f'Bearer {self.token}'}
    time.sleep(0.2)
    return requests.get(token_url, headers = authorization).json()

  #create function that will loop through the json object and retrieve Album ID, Album name, Release date, Total Tracks, Type
  def album_info(self):
    album_search_response = self.album_json_file

    album_type = [album_search_response['items'][0]['type']]
    total_tracks = [album_search_response['items'][0]['total_tracks']]
    release_date = [album_search_response['items'][0]['release_date']]
    album_id = [album_search_response['items'][1]['id']]
    available_markets = [len(album_search_response['items'][0]['available_markets'])]
    album_name = [album_search_response['items'][0]['name']]

    for item in album_search_response['items']:
      if item['name'] != album_name[-1]:
        album_type.append(item['type'])
        total_tracks.append(item['total_tracks'])
        release_date.append(item['release_date'])
        album_id.append(item['id'])
        available_markets.append(len(item['available_markets']))
        album_name.append(item['name'])
    return album_type, total_tracks, release_date, album_id, available_markets, album_name


  def track_search(self, album_id):
    token_url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    authorization = {'Authorization': f'Bearer {self.token}'}
    time.sleep(0.2)
    return requests.get(token_url, headers = authorization).json()

  def track_info(self):
    track_title,duration_ms,track_id = [],[],[]
    genre, artist_id, artist_name, artist_popularity = [],[],[],[]
    album_type, total_tracks, release_date, album_id, available_markets, album_name = [],[],[],[],[],[]
    i = 0

    for id in self.album_id:
      response = self.track_search(id)
      time.sleep(0.2)
      for item in response['items']:
        track_title.append(item['name'])
        duration_ms.append(item['duration_ms'])
        track_id.append(item['id'])

        album_type.append(self.album_type[i])
        total_tracks.append(self.total_tracks[i])
        release_date.append(self.release_date[i])
        album_id.append(self.album_id[i])
        available_markets.append(self.available_markets[i])
        album_name.append(self.album_name[i])

      i += 1
    self.track_id = track_id
    self.album_type = album_type
    self.total_tracks = total_tracks
    self.release_date = release_date
    self.album_id = album_id
    self.available_markets = available_markets
    self.album_name = album_name

    self.genre = [self.genre] * len(self.track_id)
    self.artist_id = [self.artist_id] * len(self.track_id)
    self.artist_name = [self.artist_name] * len(self.track_id)
    self.artist_popularity = [self.artist_popularity] * len(self.track_id)

    self.track_title = track_title
    self.duration_ms = duration_ms
    self.track_id = track_id
    return track_title, duration_ms, track_id


  def track_deep_search(self, track_id):
    token_url = f'https://api.spotify.com/v1/tracks/{track_id}'
    authorization = {'Authorization': f'Bearer {self.token}'}
    return requests.get(token_url, headers = authorization).json()

  def track_deep_info(self):
    popularity = []
    for id in self.track_id:
        response = self.track_deep_search(id)
        try:
            popularity.append(response['popularity'])
            time.sleep(0.2)
        except:
            popularity.append(50)
        else:
            pass
    self.track_popularity = popularity
    return popularity

In [7]:
a,b,n,c,d,e,f,g,h,i,j,k,l,m = [],[],[],[],[],[],[],[],[],[],[],[],[],[]

In [8]:
artists[0:1]

['drake']

In [9]:
for artist in artists[0:1]:
    if expires <= datetime.datetime.now():
        token, expires = authentication()
        artist_info = Artist_breakdown(artist, token)
        a += artist_info.track_title
        b += artist_info.duration_ms
        n += artist_info.track_id
        c += artist_info.artist_name
        d += artist_info.artist_id
        e += artist_info.genre
        f += artist_info.artist_popularity
        g += artist_info.album_name
        h += artist_info.release_date
        i += artist_info.album_id
        j += artist_info.total_tracks
        k += artist_info.available_markets
        l += artist_info.album_type
        m += artist_info.track_popularity
    else:
        artist_info = Artist_breakdown(artist, token)
        a += artist_info.track_title
        b += artist_info.duration_ms
        n += artist_info.track_id
        c += artist_info.artist_name
        d += artist_info.artist_id
        e += artist_info.genre
        f += artist_info.artist_popularity
        g += artist_info.album_name
        h += artist_info.release_date
        i += artist_info.album_id
        j += artist_info.total_tracks
        k += artist_info.available_markets
        l += artist_info.album_type
        m += artist_info.track_popularity
my_dictionary = {'Track Title':a, 'Track Duration (sec)':b, 'Track ID': n,
                 'Artist':c, 'Artist ID':d, 'Genre':e,
                 'Artist Popularity':f, 'Album Name':g,
                 'Release Date':h,'Album ID':i, 'Total Tracks':j,
                 'Available Markets':k,'Album Type':l,'Track Popularity':m}

drake


In [10]:
a

['Intro',
 'Falling Back',
 'Texts Go Green',
 'Currents',
 'A Keeper',
 'Calling My Name',
 'Sticky',
 'Massive',
 "Flight's Booked",
 'Overdrive',
 'Down Hill',
 'Tie That Binds',
 'Liability',
 'Jimmy Cooks (feat. 21 Savage)',
 'Champagne Poetry',
 'Papi’s Home',
 'Girls Want Girls (with Lil Baby)',
 'In The Bible (with Lil Durk & Giveon)',
 'Love All (with JAY-Z)',
 'Fair Trade (with Travis Scott)',
 'Way 2 Sexy (with Future & Young Thug)',
 'TSU',
 'N 2 Deep',
 'Pipe Down',
 'Yebba’s Heartbreak',
 'No Friends In The Industry',
 'Knife Talk (with 21 Savage ft. Project Pat)',
 '7am On Bridle Path',
 'Race My Mind',
 'Fountains (with Tems)',
 'Get Along Better',
 'You Only Live Twice (with Lil Wayne & Rick Ross)',
 'IMY2 (with Kid Cudi)',
 'F*****g Fans',
 'Deep Pockets',
 'When To Say When',
 'Chicago Freestyle (feat. Giveon)',
 'Not You Too (feat. Chris Brown)',
 'Toosie Slide',
 'Desires (with Future)',
 'Time Flies',
 'Landed',
 'D4L',
 'Pain 1993 (with Playboi Carti)',
 'Losses'

**DATAFRAME**

In [11]:
df = pd.DataFrame(my_dictionary)
df

,Track Title,Track Duration (sec),Track ID,Artist,Artist ID,Genre,Artist Popularity,Album Name,Release Date,Album ID,Total Tracks,Available Markets,Album Type,Track Popularity
0,Intro,36935,5lVmzrKaqPpPYtF7udoiNq,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,"Honestly, Nevermind",2022-06-17,2hicq6IKmDsPxZV7fJablX,14,182,album,70
1,Falling Back,266178,5zyZM1EiSqrniyqtprwxpv,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,"Honestly, Nevermind",2022-06-17,2hicq6IKmDsPxZV7fJablX,14,182,album,73
2,Texts Go Green,308820,7a5fliY9SsZENrQWQBZron,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,"Honestly, Nevermind",2022-06-17,2hicq6IKmDsPxZV7fJablX,14,182,album,72
3,Currents,157390,7IaeavdRzXYQWfu2Wt5GlU,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,"Honestly, Nevermind",2022-06-17,2hicq6IKmDsPxZV7fJablX,14,182,album,70
4,A Keeper,173097,5frM7XSby6Pj2Chs9gaKxj,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,"Honestly, Nevermind",2022-06-17,2hicq6IKmDsPxZV7fJablX,14,182,album,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Too Much,261866,5LnXWeA9tuWKI2C1H74iiJ,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,Nothing Was The Same (Deluxe),2013-01-01,2ZUFSbIkmFkGag000RWOpA,16,176,album,60
183,Pound Cake / Paris Morton Music 2,433800,4RI9eX7jNcdaQOJifn7t6z,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,Nothing Was The Same (Deluxe),2013-01-01,2ZUFSbIkmFkGag000RWOpA,16,176,album,66
184,Come Thru,236360,5Ggfa9cpkpfp5D6Rg0Yyw1,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,Nothing Was The Same (Deluxe),2013-01-01,2ZUFSbIkmFkGag000RWOpA,16,176,album,57
185,All Me,271573,5ngydCLbzCEdtu8SEOXhPU,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,94,Nothing Was The Same (Deluxe),2013-01-01,2ZUFSbIkmFkGag000RWOpA,16,176,album,60


In [ ]:
df.to_csv(f'Artist_API_Call_.csv', index = False, sep = '\t')

In [ ]:
df.info()

## PART 2: Create 3 Tables
- Artist_dataset
- Albums_dataset
- Tracks_dataset

___

<img src='img/model-schema_.jpg' />
___

In [ ]:
dat = df.copy()

In [13]:
dat.head()

,track_id,track_title,track_duration_sec,artist,artist_id,genre,artist_popularity,album_name,release_date,album_id,total_tracks,available_markets,album_type,track_popularity
0,7LSINVHqMXPt4FkTgcH6bt,Champagne Poetry,336511.0,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,93,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,183,album,39
1,6C62fl8x0vzwxPqay8twie,Papi’s Home,178623.0,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,93,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,183,album,35
2,60kRxS2TC4qb2WBPmRmfv4,Girls Want Girls (with Lil Baby),221979.0,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,93,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,183,album,49
3,7xXVDIulqmlBvDLFMfc3uX,In The Bible (with Lil Durk & Giveon),296568.0,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,93,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,183,album,36
4,7L8V5vHLo6fMbNaVI7rbhU,Love All (with JAY-Z),228461.0,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,93,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,183,album,36


In [14]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14107 entries, 0 to 14106
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   track_id            14107 non-null  object 
 1   track_title         14107 non-null  object 
 2   track_duration_sec  14107 non-null  float64
 3   artist              14107 non-null  object 
 4   artist_id           14107 non-null  object 
 5   genre               14107 non-null  object 
 6   artist_popularity   14107 non-null  int64  
 7   album_name          14107 non-null  object 
 8   release_date        14107 non-null  object 
 9   album_id            14107 non-null  object 
 10  total_tracks        14107 non-null  int64  
 11  available_markets   14107 non-null  int64  
 12  album_type          14107 non-null  object 
 13  track_popularity    14107 non-null  int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 1.5+ MB


In [15]:
artist = dat[['artist', 'artist_id', 'genre', 'artist_popularity']]
artist = artist.reindex(columns=['artist_id', 'artist', 'genre', 'artist_popularity'])
artist.drop_duplicates(subset=['artist_id'], inplace=True)
artist.reset_index(drop=True, inplace=True)
artist.head()

,artist_id,artist,genre,artist_popularity
0,3TVXtAsR1Inumwj472S9r4,Drake,canadian hip hop,93
1,50co4Is1HCEo8bhOyUWKpn,Young Thug,atl hip hop,84
2,3Nrfpe0tUJi4K4DXYWgMUX,BTS,k-pop,90
3,4dpARuHxo51G3z768sgnrY,Adele,british soul,84
4,06HL4z0CvFAxyc27GXpf02,Taylor Swift,pop,92


In [16]:
albums = dat[['artist_id','album_name', 'release_date', 'album_id', 
             'total_tracks', 'available_markets', 'album_type']]
albums = albums.reindex(columns=['album_id', 'album_name', 'artist_id', 
            'release_date', 'total_tracks', 'available_markets', 'album_type'])
albums.drop_duplicates(subset=['album_id'], inplace=True)
albums.reset_index(drop=True, inplace=True)
albums.head()

,album_id,album_name,artist_id,release_date,total_tracks,available_markets,album_type
0,6sp02aeyiwfX35xRqwNiPv,Certified Lover Boy,3TVXtAsR1Inumwj472S9r4,2021-09-03,21,183,album
1,6OQ9gBfg5EXeNAEwGSs6jK,Dark Lane Demo Tapes,3TVXtAsR1Inumwj472S9r4,2020-05-01,14,183,album
2,7dqpveMVcWgbzqYrOdkFTD,Care Package,3TVXtAsR1Inumwj472S9r4,2019-08-02,17,183,album
3,2podUJIFG8hLfFz7Kqe8yJ,So Far Gone,3TVXtAsR1Inumwj472S9r4,2019-02-14,18,181,album
4,1ATL5GLyefJaxhQzSPVrLX,Scorpion,3TVXtAsR1Inumwj472S9r4,2018-06-29,25,179,album


In [17]:
tracks = dat[['track_id', 'track_title', 'track_duration_sec', 'album_id', 'track_popularity']]
tracks = tracks.reindex(columns=['track_id', 'track_title', 'album_id', 'track_duration_sec', 'track_popularity'])
tracks.drop_duplicates(subset=['track_id'], inplace=True)
tracks.reset_index(drop=True, inplace=True)
tracks.head()

,track_id,track_title,album_id,track_duration_sec,track_popularity
0,7LSINVHqMXPt4FkTgcH6bt,Champagne Poetry,6sp02aeyiwfX35xRqwNiPv,336511.0,39
1,6C62fl8x0vzwxPqay8twie,Papi’s Home,6sp02aeyiwfX35xRqwNiPv,178623.0,35
2,60kRxS2TC4qb2WBPmRmfv4,Girls Want Girls (with Lil Baby),6sp02aeyiwfX35xRqwNiPv,221979.0,49
3,7xXVDIulqmlBvDLFMfc3uX,In The Bible (with Lil Durk & Giveon),6sp02aeyiwfX35xRqwNiPv,296568.0,36
4,7L8V5vHLo6fMbNaVI7rbhU,Love All (with JAY-Z),6sp02aeyiwfX35xRqwNiPv,228461.0,36


## **PART 3: Store Tables into PostgreSQL Database**

**CONNECT TO DATABASE**

In [18]:
host_name = 'localhost'
dbname = 'spotify_data_db'
port = '5432'
username = 'postgres' 
password = 'password'
conn = None

In [19]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [20]:
conn = connect_to_db(host_name, dbname,port, username, password)

Connected!


In [21]:
curr = conn.cursor()

## Append **ARTIST** dataframe onto database

**Create artist table**

In [22]:
def create_artist_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS artist_dataset (
                    artist_id VARCHAR(255) PRIMARY KEY NOT NULL,
                    artist VARCHAR(255) NOT NULL,
                    genre VARCHAR(255) NOT NULL,
                    artist_popularity INTEGER NOT NULL
            )""")
    curr.execute(create_table_command)

In [23]:
create_artist_table(curr)

In [24]:
conn.commit()

**Check if row item in artist dataframe exists on database** <br>
If exists, update values across entire line item<br>
If not, append the line item onto a new placeholder dataframe

In [25]:
def update_artist_table(curr,df):
    tmp_df = pd.DataFrame(columns=['artist_id', 'artist', 'genre', 'artist_popularity'])
    for i, row in df.iterrows():
        if check_if_artist_exists_on_db(curr, row['artist_id']):
            update_artist_row(curr,row['artist_id'],row['artist'],row['genre'],row['artist_popularity'])
        else: # The track doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [26]:
def check_if_artist_exists_on_db(curr, artist_id): 
    query = ("""SELECT artist_id FROM artist_dataset WHERE artist_id = %s""")

    curr.execute(query, (artist_id,))
    return curr.fetchone() is not None

In [27]:
def update_artist_row(curr, artist_id, artist, genre, artist_popularity):
    query = ("""UPDATE artist_dataset
            SET artist_id = %s,
                artist = %s,
                genre = %s,
                artist_popularity = %s
            WHERE artist_id = %s;""")
    vars_to_update = (artist_id, artist, genre, artist_popularity, artist_id)
    curr.execute(query, vars_to_update)

In [28]:
new_artist_df = update_artist_table(curr,artist)
conn.commit()

**Insert values from placeholder dataframe into artist_dataset database**

In [29]:
def append_from_artist_phdf_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_artist_table(curr,row['artist_id'],row['artist'],row['genre'],row['artist_popularity'])

In [30]:
def insert_into_artist_table(curr, artist_id, artist, genre, artist_popularity):
    insert_into_tracks = ("""INSERT INTO artist_dataset (artist_id,artist,genre,artist_popularity)
    VALUES(%s,%s,%s,%s)
    ON CONFLICT (artist_id)
    DO NOTHING;""")
    row_to_insert = (artist_id, artist, genre, artist_popularity)
    curr.execute(insert_into_tracks, row_to_insert)

In [32]:
append_from_artist_phdf_to_db(curr, new_artist_df)
conn.commit()

## Append **ALBUMS** dataframe onto database

**Create albums table**

In [33]:
def create_albums_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS albums_dataset (
                    album_id VARCHAR(255) PRIMARY KEY NOT NULL,
                    album_name VARCHAR(255) NOT NULL,
                    artist_id VARCHAR(255) REFERENCES artist_dataset(artist_id) NOT NULL,
                    release_date VARCHAR(50) NOT NULL,
                    total_tracks INTEGER NOT NULL,
                    available_markets INTEGER NOT NULL,
                    album_type VARCHAR(255) NOT NULL
            )""")
    curr.execute(create_table_command)

In [34]:
create_albums_table(curr)

In [35]:
conn.commit()

**Check if row item in albums dataframe exists on database** <br>
If exists, update values across entire line item<br>
If not, append the line item onto a new placeholder dataframe

In [36]:
def update_albums_table(curr,df):
    tmp_df = pd.DataFrame(columns=['album_id', 'album_name', 'artist_id', 'release_date', 'total_tracks',
       'available_markets', 'album_type'])
    for i, row in df.iterrows():
        if check_if_album_exists_on_db(curr, row['album_id']):
            update_album_row(curr,row['album_id'],row['album_name'],row['artist_id'],row['release_date'],
                             row['total_tracks'],row['available_markets'],row['album_type'])
        else: 
            tmp_df = tmp_df.append(row)

    return tmp_df

In [37]:
def check_if_album_exists_on_db(curr, album_id): 
    query = ("""SELECT album_id FROM albums_dataset WHERE album_id = %s""")

    curr.execute(query, (album_id,))
    return curr.fetchone() is not None

In [38]:
def update_album_row(curr, album_id, album_name, artist_id, release_date, total_tracks, 
                     available_markets, album_type):
    query = ("""UPDATE albums_dataset
            SET album_id = %s,
                album_name = %s,
                artist_id = %s,
                release_date = %s,
                total_tracks = %s,
                available_markets = %s,
                album_type = %s
            WHERE artist_id = %s;""")
    vars_to_update = (album_id, album_name, artist_id, release_date, total_tracks, 
                     available_markets, album_type)
    curr.execute(query, vars_to_update)

In [39]:
new_albums_df = update_albums_table(curr,albums)
conn.commit()

**Insert values from placeholder dataframe into album_dataset database**

In [40]:
albums.columns

Index(['album_id', 'album_name', 'artist_id', 'release_date', 'total_tracks',
       'available_markets', 'album_type'],
      dtype='object')

In [41]:
def append_from_album_phdf_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_album_table(curr,row['album_id'],row['album_name'],row['artist_id'],row['release_date'],
                             row['total_tracks'],row['available_markets'],row['album_type'])

In [42]:
def insert_into_album_table(curr, album_id, album_name, artist_id, release_date, total_tracks, 
                     available_markets, album_type):
    insert_into_tracks = ("""INSERT INTO albums_dataset (album_id, album_name, artist_id, release_date, total_tracks, 
                     available_markets, album_type)
    VALUES(%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (album_id)
    DO NOTHING;""")
    row_to_insert = (album_id, album_name, artist_id, release_date, total_tracks, 
                     available_markets, album_type)
    curr.execute(insert_into_tracks, row_to_insert)

In [43]:
append_from_album_phdf_to_db(curr, new_albums_df)
conn.commit()

## Append **TRACKS** dataframe onto database

**Create tracks table**

In [44]:
def create_tracks_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS tracks_dataset (
                    track_id VARCHAR(255) PRIMARY KEY,
                    track_title VARCHAR(255) NOT NULL,
                    track_duration_sec NUMERIC NOT NULL,
                    album_id VARCHAR(255) REFERENCES albums_dataset(album_id) NOT NULL,
                    track_popularity INTEGER NOT NULL
            )""")
    curr.execute(create_table_command)

In [45]:
create_tracks_table(curr)

In [46]:
conn.commit()

**Check if row item in tracks dataframe exists on database** <br>
If exists, update values across entire line item<br>
If not, append the line item onto a new placeholder dataframe

In [47]:
tracks.columns

Index(['track_id', 'track_title', 'album_id', 'track_duration_sec',
       'track_popularity'],
      dtype='object')

In [48]:
def update_tracks_table(curr,df):
    tmp_df = pd.DataFrame(columns=['track_id', 'track_title', 'album_id', 'track_duration_sec', 'track_popularity'])
    for i, row in df.iterrows():
        if check_if_track_exists_on_db(curr, row['track_id']):
            update_track_row(curr,row['track_id'],row['track_title'],row['album_id'],
                             row['track_duration_sec'],row['track_popularity'])
        else: 
            tmp_df = tmp_df.append(row)

    return tmp_df

In [49]:
def check_if_track_exists_on_db(curr, track_id): 
    query = ("""SELECT track_id FROM tracks_dataset WHERE track_id = %s""")

    curr.execute(query, (track_id,))
    return curr.fetchone() is not None

In [50]:
def update_track_row(curr, track_id, track_title, album_id, track_duration_sec, track_popularity):
    query = ("""UPDATE tracks_dataset
            SET track_id = %s,
                track_title = %s,
                album_id = %s,
                track_duration_sec = %s,
                track_popularity = %s
            WHERE track_id = %s;""")
    vars_to_update = (track_id, track_title, album_id, track_duration_sec, track_popularity)
    curr.execute(query, vars_to_update)

In [51]:
new_tracks_df = update_tracks_table(curr,tracks)
conn.commit()

**Insert values from placeholder dataframe into tracks_dataset database**

In [52]:
def append_from_tracks_phdf_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_tracks_table(curr,row['track_id'],row['track_title'],row['album_id'],
                             row['track_duration_sec'],row['track_popularity'])

In [53]:
def insert_into_tracks_table(curr, track_id, track_title, album_id, track_duration_sec, track_popularity):
    insert_into_tracks = ("""INSERT INTO tracks_dataset (track_id, track_title, album_id, 
    track_duration_sec, track_popularity)
    VALUES(%s,%s,%s,%s,%s)
    ON CONFLICT (track_id)
    DO NOTHING;""")
    row_to_insert = (track_id, track_title, album_id, track_duration_sec, track_popularity)
    curr.execute(insert_into_tracks, row_to_insert)

In [54]:
append_from_tracks_phdf_to_db(curr, new_tracks_df)
conn.commit()

In [ ]:
#end